In [ ]:
import numpy as np
import pandas as pd
import time
from numba import njit
from tqdm.notebook import tqdm, trange # For Jupyter notebooks

# =======================================================
# --- Functions ---
# =======================================================
@njit
def run_particle_system(gamma, beta, v, x0, T, K, N):
    h = T / K
    sqrt_h = np.sqrt(h)
    X = np.full(N, x0, dtype=np.float64)
    for _ in range(K):
        dW = sqrt_h * np.random.randn(N)
        mean_X = np.mean(X)
        X += (gamma * X + beta * mean_X) * h + v * dW
    return np.mean(X)


def get_mean_X_values(gamma, beta, v, x0, T, K, N, num_runs):
    mean_X_values = np.zeros(num_runs)

    for run in trange(num_runs, desc="Monte Carlo Runs", leave=False, dynamic_ncols=True):
        mean_X_values[run] = run_particle_system(gamma, beta, v, x0, T, K, N)

    return mean_X_values


def calculate_first_moment(gamma, beta, x0, T, K):
    h = T / K
    first_moment = (1 + (gamma + beta) * h)**K * x0
    return first_moment


def calculate_precision(empirical_mean, num_runs):
    variance = np.var(empirical_mean, ddof=1) # ddof=1 for unbiased estimatior of variance
    precision = 1.96 * np.sqrt(variance / num_runs)
    return precision


# =======================================================
# --- Parameters ---
# =======================================================
gamma = -0.5
beta = 0.8
v_squared = 0.5
v = np.sqrt(v_squared)
x0 = 1
T = 1
K = 50
num_runs = 5 * 10**6 # for quick testing, paper used 5 * 10**6
N_values = [20, 40, 80, 160, 320]


# =======================================================
# --- Main Simulation Loop ---
# =======================================================
first_moment = calculate_first_moment(
    gamma, beta, x0, T, K
)
print(f"Closed-form discretized value for E[X_T^(1,N,h)]: {first_moment:.5f}\n")

table_data = []

for N in tqdm(N_values, desc="Loop over N", leave=False, dynamic_ncols=True):
    print(f"\nRunning for N = {N} particles...")

    # Get all terminal values from num_runs simulations
    start_X_time = time.time()
    mean_X_values = get_mean_X_values(
        gamma, beta, v, x0, T, K, N, num_runs
    )
    end_X_time = time.time()
    print(f"Time taken to compute X values for N={N}: {end_X_time - start_X_time:.2f} seconds")

    estimated_first_moment = np.mean(mean_X_values)
    difference = estimated_first_moment - first_moment
    precision = calculate_precision(mean_X_values, num_runs)

    table_data.append({
        "Nb. particles": N,
        "Estimated first moment": estimated_first_moment,
        "Difference": difference,
        "Precision": precision
    })


# =======================================================
# --- Table Generation ---
# =======================================================
df_table1 = pd.DataFrame(table_data)

print(f"\n\nClosed-form discretized value (reference for 'Difference'): {first_moment:.5f}\n")
desired_columns = ["Nb. particles", "Estimated first moment", "Difference", "Precision"]
df_print = df_table1[desired_columns].copy()
df_print["Estimated first moment"] = df_print["Estimated first moment"].map('{:.5f}'.format)
df_print["Difference"] = df_print["Difference"].map('{:.5f}'.format) # Adjust format if very small values
df_print["Precision"] = df_print["Precision"].map('{:.5f}'.format)
print(df_print.to_string(index=False))

Closed-form discretized value for E[X_T^(1,N,h)]: 1.34865



Loop over N:   0%|          | 0/5 [00:00<?, ?it/s]


Running for N = 20 particles...


Monte Carlo Runs:   0%|          | 0/5000000 [00:00<?, ?it/s]

Time taken to compute X values for N=20: 171.17 seconds

Running for N = 40 particles...


...

Monte Carlo Runs:   0%|          | 0/5000000 [00:00<?, ?it/s]

Time taken to compute X values for N=160: 987.22 seconds

Running for N = 320 particles...


Monte Carlo Runs:   0%|          | 0/5000000 [00:00<?, ?it/s]

Time taken to compute X values for N=320: 1893.07 seconds


Closed-form discretized value (reference for 'Difference'): 1.34865

 Nb. particles Estimated first moment Difference Precision
            20                1.34882    0.00017   0.00016
            40                1.34870    0.00006   0.00011
            80                1.34859   -0.00006   0.00008
           160                1.34866    0.00001   0.00006
           320                1.34863   -0.00002   0.00004


'\nprint("\n\n--- Estimated Precision if num_runs were 5 * 10^6 ---")\ndf_scaled_precision = df_table1[["Nb. particles", "Precision"]].copy() # Use original float precision\n\ntarget_num_runs = 5 * 10**6\nscaling_factor = np.sqrt(num_runs / target_num_runs)\n\ndf_scaled_precision["Scaled Precision (for 5e6 runs)"] = df_scaled_precision["Precision"] * scaling_factor\ndf_scaled_precision_print = df_scaled_precision[["Nb. particles", "Scaled Precision (for 5e6 runs)"]].copy()\ndf_scaled_precision_print["Scaled Precision (for 5e6 runs)"] =     df_scaled_precision_print["Scaled Precision (for 5e6 runs)"].map(\'{:.5f}\'.format)\nprint(df_scaled_precision_print.to_string(index=False))\n'